In [1]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 25.6 MB/s eta 0:00:00


# E-Commerce Recommendation System Model Training

This notebook builds and trains a recommendation model for our Django e-commerce application.

## Approach:
1. Use a dataset of past purchases to build a recommendation model.
2. Use a machine learning approach like collaborative filtering (ALS from implicit library) or content-based filtering (TF-IDF, cosine similarity).
3. Save trained models as pickle files

## Import the dependencies

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares
import joblib
import re

## Load the dataset

In [25]:
# Load sample transaction data
df = pd.read_csv("/content/dummy/ecommerce_product_dataset.csv")

## View the columns of the dataset

In [26]:
df.columns

Index(['ProductID', 'ProductName', 'Category', 'Price', 'Rating', 'NumReviews',
       'StockQuantity', 'Discount', 'Sales', 'DateAdded'],
      dtype='object')

# View the first 5 rows of the dataset

In [27]:
df.head()

,ProductID,ProductName,Category,Price,Rating,NumReviews,StockQuantity,Discount,Sales,DateAdded
0,1,Headphones,Electronics,400.31,1.7,3772,20,0.08,466,2023-11-08
1,2,Headphones,Electronics,235.03,2.3,2919,663,0.33,1332,2023-09-26
2,3,Smartwatch,Electronics,417.90,1.8,1184,459,0.31,252,2023-10-18
3,4,Smartphone,Electronics,152.70,3.4,2047,475,0.49,1806,2023-12-03
4,5,Laptop,Electronics,394.74,1.8,1267,831,0.23,1508,2023-08-08


## View a column from the dataset

In [32]:
df["ProductID"]

,ProductID
0,1
1,2
2,3
3,4
4,5
...,...
995,996
996,997
997,998
998,999


## Create a sparse matrix

In [28]:
# Create an item-user sparse matrix
user_item_matrix = df.pivot(index="ProductID", columns="ProductName", values="Sales").fillna(0)
sparse_item_user = sparse.csr_matrix(user_item_matrix)

In [29]:
sparse_item_user

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 999 stored elements and shape (1000, 100)>

## Train the ALS model

In [30]:
# Train the ALS model
model = AlternatingLeastSquares(factors=50, regularization=0.01, iterations=15)
model.fit(sparse_item_user.T)  # Transpose so it’s user-item

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.007766246795654297 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

## Save the recommendation model as a pickle file

In [31]:
# Save the model
joblib.dump(model, "recommendation_model.pkl")

['recommendation_model.pkl']

# Testing the recommendation model pickle file

In [33]:
# Load the trained recommendation model
model = joblib.load("/content/recommendation_model.pkl")

# Check if the model loaded correctly
print(type(model))  # Should print the model type (e.g., `implicit.cpu.als.AlternatingLeastSquares`)

<class 'implicit.cpu.als.AlternatingLeastSquares'>


## Use an example of a product id

In [38]:
# Example: Get recommendations for product with ID 101
product_id = 278
num_recommendations = 5

# Generate similar item recommendations
recommendations, scores = model.similar_items(product_id, num_recommendations)



In [39]:
recommendations

array([251, 247, 243, 256, 246], dtype=int32)

In [40]:
scores

array([1.0000001, 1.0000001, 1.0000001, 1.       , 1.       ],
      dtype=float32)

## Displays similar products

In [41]:
# Display recommendations
print(f"Recommendations for Product ID {product_id}:")
for rec_id, score in zip(recommendations, scores):
    print(f"Product ID: {rec_id}, Similarity Score: {score}")


Recommendations for Product ID 278:
Product ID: 251, Similarity Score: 1.0000001192092896
Product ID: 247, Similarity Score: 1.0000001192092896
Product ID: 243, Similarity Score: 1.0000001192092896
Product ID: 256, Similarity Score: 1.0
Product ID: 246, Similarity Score: 1.0
